In [4]:
!pip3 install pillow-heif
from PIL import Image
import math
from pillow_heif import register_heif_opener
register_heif_opener()

In [ ]:
# @title
def calculate_scaling_factor(original_width, original_height, target_width, target_height):
    original_area = original_width * original_height
    target_area = target_width * target_height
    return math.sqrt(target_area / original_area)

def rescale_image(image_path, output_path, target_width = 2048, target_height = 1536):
  with Image.open(image_path) as img:
      original_width, original_height = img.size
      scaling_factor = calculate_scaling_factor(original_width, original_height, target_width, target_height)
      new_width = int(original_width * scaling_factor)
      new_height = int(original_height * scaling_factor)
      img_resized = img.resize((new_width, new_height))
      img_resized.save(output_path)

  print(f"Image has been downsampled and saved to {output_path}")

# image_path = '/content/downsampled_mega.jpg'  # Replace with your image path
# output_path = '/content/downsampled_mega_rescaled.jpg'
# target_width = 2048
# target_height = 1536

In [ ]:
# @title
# prompt: split my image into 224x224 pixel segments and show a grid of each segment

import matplotlib.pyplot as plt

# Load the downsampled image
image_path = "/content/downsampled/biggest_clovers_device.jpg"
img = Image.open(image_path)

# Define the grid dimensions
grid_width = 224
grid_height = 224

# Calculate the number of rows and columns in the grid
rows = math.ceil(img.height / grid_height)
cols = math.ceil(img.width / grid_width)

# Create a figure with subplots
fig, axs = plt.subplots(rows, cols, figsize=(100, 100))

# Iterate over each row and column
for i in range(rows):
    for j in range(cols):
        # Calculate the coordinates of the current segment
        x = j * grid_width
        y = i * grid_height
        w = grid_width
        h = grid_height

        # Check if the segment is within the image boundaries
        if x + w > img.width:
            w = img.width - x
        if y + h > img.height:
            h = img.height - y

        # Crop the segment from the image
        segment = img.crop((x, y, x + w, y + h))

        # Display the segment in the corresponding subplot
        axs[i, j].imshow(segment)
        axs[i, j].set_xticks([])
        axs[i, j].set_yticks([])

# Show the plot
plt.show()


In [75]:
#@title Preprocess and Split

import os
import math
from PIL import Image

def calculate_scaling_factor(original_width, original_height, target_width, target_height):
    original_area = original_width * original_height
    target_area = target_width * target_height
    return math.sqrt(target_area / original_area)

def preprocess_image(image_path, output_dir, unique_id, target_width=2048, target_height=1536):
    with Image.open(image_path) as img:
        original_width, original_height = img.size
        scaling_factor = calculate_scaling_factor(original_width, original_height, target_width, target_height)
        new_width = int(original_width * scaling_factor)
        new_height = int(original_height * scaling_factor)
        img_resized = img.resize((new_width, new_height), Image.LANCZOS)

        # Convert to RGB and save as BMP without metadata
        img_resized = img_resized.convert("RGB")
        output_path = os.path.join(output_dir, f"{unique_id}.bmp")
        img_resized.save(output_path, format='BMP')

    print(f"Image {image_path} has been preprocessed and saved as {output_path}")
    return output_path

def split_image(image_path, output_dir, unique_id, segment_size=224, stride=194):
    with Image.open(image_path) as img:
        img_width, img_height = img.size
        segment_id = unique_id

        for x in range(0, img_width, stride):
            for y in range(0, img_height, stride):
                # Calculate the boundaries of the segment
                adjusted_x = min(x, img_width - segment_size)
                adjusted_y = min(y, img_height - segment_size)

                # Create the segment
                segment = img.crop((adjusted_x, adjusted_y, adjusted_x + segment_size, adjusted_y + segment_size))

                # Convert to RGB and save as BMP without metadata
                segment = segment.convert("RGB")
                segment_output_path = os.path.join(output_dir, f"{unique_id}_{adjusted_x}_{adjusted_y}.bmp")
                segment.save(segment_output_path, format='BMP')

                segment_id += 1
        return segment_id

def preprocess_and_split_images_in_directory(input_directory, preprocessed_output_directory, segments_output_directory, target_width=2048, target_height=1536, segment_size=224, stride=194):
    unique_id = 0
    for root, _, files in os.walk(input_directory):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'heic')):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, input_directory)

                # Preprocess image
                preprocessed_output_dir = os.path.join(preprocessed_output_directory, relative_path)
                os.makedirs(preprocessed_output_dir, exist_ok=True)
                preprocessed_image_path = preprocess_image(input_path, preprocessed_output_dir, unique_id, target_width, target_height)

                # Split image into segments
                segments_output_dir = os.path.join(segments_output_directory, relative_path)
                os.makedirs(segments_output_dir, exist_ok=True)
                split_image(preprocessed_image_path, segments_output_dir, unique_id, segment_size, stride)

                unique_id += 1

# Main execution
input_directory = '/content/test_images'  # Replace with your input directory path
preprocessed_output_directory = '/content/preprocessed'  # Directory for preprocessed images
segments_output_directory = '/content/split_and_preprocessed'  # Directory for segmented images

# Preprocess images and split them into segments
preprocess_and_split_images_in_directory(input_directory, preprocessed_output_directory, segments_output_directory, target_width=2048, target_height=1536, segment_size=224, stride=194)

print(f"Preprocessed images have been saved to {preprocessed_output_directory}")
print(f"Segmented images have been saved to {segments_output_directory}")

Image /content/test_images/high_zoom7.jpg has been preprocessed and saved as /content/preprocessed/./0.bmp
Image /content/test_images/high_zoom6.JPG has been preprocessed and saved as /content/preprocessed/./1.bmp
Image /content/test_images/d3/high_zoom5.JPG has been preprocessed and saved as /content/preprocessed/d3/2.bmp
Image /content/test_images/d3/high_zoom4.jpeg has been preprocessed and saved as /content/preprocessed/d3/3.bmp
Image /content/test_images/d1/high_zoom1.JPG has been preprocessed and saved as /content/preprocessed/d1/4.bmp
Image /content/test_images/d1/d2/high_zoom2.heic has been preprocessed and saved as /content/preprocessed/d1/d2/5.bmp
Image /content/test_images/d1/d2/high_zoom3.heic has been preprocessed and saved as /content/preprocessed/d1/d2/6.bmp
Preprocessed images have been saved to /content/preprocessed
Segmented images have been saved to /content/split_and_preprocessed


In [76]:
#@title Reconstruct

import os
from PIL import Image
from collections import defaultdict

def reconstruct_images_from_segments(segments_directory, output_directory, segment_size=224):
    # Dictionary to store image segments by their original image ID
    segments_dict = defaultdict(list)

    # Collect segments
    for root, _, files in os.walk(segments_directory):
        for file in files:
            if file.lower().endswith(('bmp', 'jpg', 'jpeg', 'gif')):
                file_path = os.path.join(root, file)
                parts = file.split('_')
                if len(parts) == 3:
                    image_id = int(parts[0])
                    x = int(parts[1])
                    y = int(parts[2].split('.')[0])
                    segments_dict[image_id].append((x, y, file_path))

    # Reconstruct images
    for image_id, segments in segments_dict.items():
        # Determine the original image dimensions
        max_x = max(segment[0] for segment in segments) + segment_size
        max_y = max(segment[1] for segment in segments) + segment_size

        # Create a new blank image
        full_image = Image.new('RGB', (max_x, max_y))

        # Paste the segments into the full image
        for x, y, file_path in segments:
            segment = Image.open(file_path)
            full_image.paste(segment, (x, y))

        # Save the reconstructed image using default settings
        relative_path = os.path.relpath(file_path, segments_directory)
        original_dir = os.path.dirname(relative_path)
        output_path = os.path.join(output_directory, original_dir)
        os.makedirs(output_path, exist_ok=True)
        full_image.save(os.path.join(output_path, f'{image_id}.bmp'), format='BMP')

    print(f"Reconstructed images have been saved to {output_directory}")

# Main execution
segments_directory = '/content/split_and_preprocessed'  # Directory containing the segmented images
reconstructed_output_directory = '/content/reconstructed_images'  # Directory to save reconstructed images

# Reconstruct images from segments
reconstruct_images_from_segments(segments_directory, reconstructed_output_directory)

print(f"Reconstructed images have been saved to {reconstructed_output_directory}")

Reconstructed images have been saved to /content/reconstructed_images
Reconstructed images have been saved to /content/reconstructed_images


In [77]:
#@title Verify Reconstructed == Preprocessed

def compare_directories(dir1, dir2):
    for root, _, files in os.walk(dir1):
        for file in files:
            preprocessed_file_path = os.path.join(root, file)
            relative_path = os.path.relpath(preprocessed_file_path, dir1)
            reconstructed_file_path = os.path.join(dir2, relative_path)

            if not os.path.exists(reconstructed_file_path):
                print(f"File {reconstructed_file_path} does not exist in {dir2}")
                return False

    temp = dir1
    dir1 = dir2
    dir2 = temp

    for root, _, files in os.walk(dir1):
      for file in files:
          preprocessed_file_path = os.path.join(root, file)
          relative_path = os.path.relpath(preprocessed_file_path, dir1)
          reconstructed_file_path = os.path.join(dir2, relative_path)

          print(f"Checking: {relative_path}...")

          if not os.path.exists(reconstructed_file_path):
              print(f"File {reconstructed_file_path} does not exist in {dir2}")
              return False

          preprocessed_img = Image.open(preprocessed_file_path)
          reconstructed_img = Image.open(reconstructed_file_path)

          preprocessed_data = list(preprocessed_img.getdata())
          reconstructed_data = list(reconstructed_img.getdata())

          if preprocessed_data != reconstructed_data:
              print(f"Images {preprocessed_file_path} and {reconstructed_file_path} do not match")
              differences = [(i, v1, v2) for i, (v1, v2) in enumerate(zip(preprocessed_data, reconstructed_data)) if v1 != v2]
              for index, v1, v2 in differences:
                  print(f"Index {index}: Preprocessed value {v1}, Reconstructed value {v2}")
              return False

    print("All images match exactly")
    return True

# Compare directories to ensure they are identical
preprocessed_dir = preprocessed_output_directory  # Directory containing the preprocessed images
reconstructed_dir = reconstructed_output_directory  # Directory containing the reconstructed images

if compare_directories(preprocessed_dir, reconstructed_dir):
    print("Test passed: Preprocessed and reconstructed directories are identical")
else:
    print("Test failed: Preprocessed and reconstructed directories differ")

Checking: 0.bmp...
Checking: 1.bmp...
Checking: d3/2.bmp...
Checking: d3/3.bmp...
Checking: d1/4.bmp...
Checking: d1/d2/6.bmp...
Checking: d1/d2/5.bmp...
All images match exactly
Test passed: Preprocessed and reconstructed directories are identical


In [73]:
!rm -rf /content/rescaled /content/split_and_rescaled /content/reconstructed_images /content/extracted_segments /content/preprocessed /content/split_and_preprocessed

In [74]:
!rm -rf /content/reconstructed_images

In [5]:
!unzip /content/test_images.zip

Archive:  /content/test_images.zip
   creating: test_images/d1/
   creating: test_images/d1/d2/
  inflating: test_images/d1/d2/high_zoom2.heic  
  inflating: test_images/d1/d2/high_zoom3.heic  
  inflating: test_images/d1/high_zoom1.JPG  
   creating: test_images/d3/
  inflating: test_images/d3/high_zoom4.jpeg  
  inflating: test_images/d3/high_zoom5.JPG  
  inflating: test_images/high_zoom6.JPG  
  inflating: test_images/high_zoom7.jpg  


biggest clovers: high_zoom3